In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyarrow.parquet as pq
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/04 16:20:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2022-01.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  357M  100  357M    0     0  11.8M      0  0:00:30  0:00:30 --:--:-- 11.6M 10.7M      0  0:00:33  0:00:12  0:00:21 11.8M      0  0:00:32  0:00:13  0:00:19 11.8M1.8M      0  0:00:30  0:00:24  0:00:06 12.8M


In [5]:
pqfile = pq.ParquetFile('./fhvhv_tripdata_2022-01.parquet')

In [6]:
df = next(pqfile.iter_batches(100)).to_pandas()

In [13]:
schema = types.StructType([types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('originating_base_num', types.StringType(), True),
    types.StructField('request_datetime', types.TimestampType(), True),
    types.StructField('on_scene_datetime', types.TimestampType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('trip_miles', types.DoubleType(), True),
    types.StructField('trip_time', types.IntegerType(), True),
    types.StructField('base_passenger_fare', types.DoubleType(), True),
    types.StructField('tolls', types.DoubleType(), True),
    types.StructField('bcf', types.DoubleType(), True),
    types.StructField('sales_tax', types.DoubleType(), True),
    types.StructField('congestion_surcharge', types.DoubleType(), True),
    types.StructField('airport_fee', types.DoubleType(), True),
    types.StructField('tips', types.DoubleType(), True),
    types.StructField('driver_pay', types.DoubleType(), True),
    types.StructField('shared_request_flag', types.StringType(), True),
    types.StructField('shared_match_flag', types.StringType(), True),
    types.StructField('access_a_ride_flag', types.StringType(), True),
    types.StructField('wav_request_flag', types.StringType(), True),
    types.StructField('wav_match_flag', types.StringType(), True)]
)

In [21]:
spark.conf.set('spark.sql.parquet.enableVectorizedReader', 'true')

In [24]:
df = spark.read \
    .parquet('./fhvhv_tripdata_2022-01.parquet')

In [25]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B03404', originating_base_num='B03404', request_datetime=datetime.datetime(2022, 1, 1, 8, 5, 31), on_scene_datetime=datetime.datetime(2022, 1, 1, 8, 5, 40), pickup_datetime=datetime.datetime(2022, 1, 1, 8, 7, 24), dropoff_datetime=datetime.datetime(2022, 1, 1, 8, 18, 28), PULocationID=170, DOLocationID=161, trip_miles=1.18, trip_time=664, base_passenger_fare=24.9, tolls=0.0, bcf=0.75, sales_tax=2.21, congestion_surcharge=2.75, airport_fee=0.0, tips=0.0, driver_pay=23.03, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B03404', originating_base_num='B03404', request_datetime=datetime.datetime(2022, 1, 1, 8, 19, 27), on_scene_datetime=datetime.datetime(2022, 1, 1, 8, 22, 8), pickup_datetime=datetime.datetime(2022, 1, 1, 8, 22, 32), dropoff_datetime=datetime.datetime(2022, 1, 1, 8, 30, 12), PULocationID=237, DOLoca

In [27]:
df = df.repartition(24)

In [28]:
df.write.parquet('fhvhv/2022/01/')

22/08/04 16:04:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/08/04 16:04:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/08/04 16:04:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/08/04 16:04:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/08/04 16:04:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/08/04 16:04:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/08/04 16:04:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/08/04 16:04:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [29]:
!ls -lh fhvhv/2022/01

total 1330176
-rw-r--r--  1 jchien  staff     0B  8  4 16:04 _SUCCESS
-rw-r--r--  1 jchien  staff    27M  8  4 16:04 part-00000-91d0087d-53f2-4b28-8919-871576adba84-c000.snappy.parquet
-rw-r--r--  1 jchien  staff    27M  8  4 16:04 part-00001-91d0087d-53f2-4b28-8919-871576adba84-c000.snappy.parquet
-rw-r--r--  1 jchien  staff    27M  8  4 16:04 part-00002-91d0087d-53f2-4b28-8919-871576adba84-c000.snappy.parquet
-rw-r--r--  1 jchien  staff    27M  8  4 16:04 part-00003-91d0087d-53f2-4b28-8919-871576adba84-c000.snappy.parquet
-rw-r--r--  1 jchien  staff    27M  8  4 16:04 part-00004-91d0087d-53f2-4b28-8919-871576adba84-c000.snappy.parquet
-rw-r--r--  1 jchien  staff    27M  8  4 16:04 part-00005-91d0087d-53f2-4b28-8919-871576adba84-c000.snappy.parquet
-rw-r--r--  1 jchien  staff    27M  8  4 16:04 part-00006-91d0087d-53f2-4b28-8919-871576adba84-c000.snappy.parquet
-rw-r--r--  1 jchien  staff    27M  8  4 16:04 part-00007-91d0087d-53f2-4b28-8919-871576adba84-c000.snappy.parquet
-rw-r--r--

## 2

In [3]:
df = spark.read.parquet('./fhvhv/2022/01/')

In [4]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [13]:
 df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show(5)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2022-01-13 21:08:25|2022-01-13 21:25:31|         162|         168|
|2022-01-07 02:40:09|2022-01-07 02:54:41|         225|          17|
|2022-01-03 05:50:27|2022-01-03 05:57:17|          90|         100|
|2022-01-08 10:47:49|2022-01-08 11:02:23|         158|         233|
|2022-01-30 11:07:55|2022-01-30 11:19:38|         148|         249|
+-------------------+-------------------+------------+------------+
only showing top 5 rows



In [14]:
from pyspark.sql import functions as F

In [17]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [20]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/d4c| 2022-01-13|  2022-01-13|         162|         168|
|  e/d4c| 2022-01-07|  2022-01-07|         225|          17|
|  e/d4c| 2022-01-03|  2022-01-03|          90|         100|
|  e/d4c| 2022-01-08|  2022-01-08|         158|         233|
|  e/d4c| 2022-01-30|  2022-01-30|         148|         249|
|  e/d4c| 2022-02-01|  2022-02-01|          52|         255|
|  e/d4c| 2022-02-01|  2022-02-01|         182|         242|
|  e/d4e| 2022-01-16|  2022-01-16|         231|         249|
|  e/d4e| 2022-01-01|  2022-01-01|         230|         164|
|  e/d4c| 2022-01-05|  2022-01-05|          79|         162|
|  e/d4c| 2022-01-05|  2022-01-05|         230|          13|
|  e/d4e| 2022-01-17|  2022-01-17|          89|         211|
|  e/d4e| 2022-01-02|  2022-01-02|          80|         188|
|  e/d4c| 2022-01-05|  2